Benchmarking RMSF analysis speed using cloud & disk reading

First perform the steps in `benchmark_setup.ipynb`

Steps:

1. Open a `zarr.Group` object from the trajectory stored on disk
2. Open one from the trajectory uploaded to an AWS S3 bucket
3. Create an `mda.Universe` object for both zarr groups and one for the original .xtc trajectory
4. Perform the RMSF analysis for each `mda.Universe`, time, and compare results